In [1]:
from bs4 import BeautifulSoup
import requests
import sqlite3
from sqlite3 import Error

import pandas as pd

import re

In [2]:
df = pd.read_csv('uk-parkruns.csv')

In [3]:
df = df.set_index('Parkrun')

In [4]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn

In [5]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [6]:
conn = create_connection("parkruns.db")

sql_create_parkrun_table = """ CREATE TABLE IF NOT EXISTS parkruns (
                                    id integer PRIMARY KEY,
                                    parkrun_name text NOT NULL UNIQUE,
                                    parkrun_country text,
                                    parkrun_location text,
                                    parkrun_latitude float,
                                    parkrun_longitude float
                                ); """

if conn is not None:
    create_table(conn, sql_create_parkrun_table)

else:
    print("Error! cannot create the database connection.")

In [7]:
def create_parkrun(conn, parkrun_name, parkrun_country, parkrun_location, parkrun_latitude, parkrun_longitude):
    """
    Create a new project into the parkruns table
    :param conn:
    :param parkrun:
    :return: project id
    """
    sql = ''' INSERT INTO parkruns(parkrun_name, parkrun_country, parkrun_location, parkrun_latitude, parkrun_longitude)
              VALUES(?,?,?,?,?) '''
    cur = conn.cursor()
    
    # When you are inserting one column value into each row, insert value like this so is interpreted as a tuple and not as an input sequence
    try:
        cur.execute(sql, (parkrun_name, parkrun_country, parkrun_location, parkrun_latitude, parkrun_longitude))
        conn.commit()
    except:
        print('A parkrun with that name is already in database')
    return cur.lastrowid

In [8]:
parkrun_country = 'UK'
for parkrun_name in df.index:
    parkrun_location =  df.loc[parkrun_name]['Location']
    parkrun_latitude =  df.loc[parkrun_name]['Latitude']
    parkrun_longitude =  df.loc[parkrun_name]['Longitude']
    create_parkrun(conn, parkrun_name, parkrun_country, parkrun_location, parkrun_latitude, parkrun_longitude)

In [10]:
def add_parkruns_from_country_to_database(parkrun_country, country_url, wiki_bool):
    page = requests.get(country_url)

    soup = BeautifulSoup(page.content, "html.parser")

    parkrun_names = [i.text for i in soup.select('td:nth-child(1)')]
    parkrun_locations = [i.text for i in soup.select('td:nth-child(2)')]
    if wiki_bool == False:
        parkrun_countries = [i.text for i in soup.select('td:nth-child(3)')]

    for i in range(len(parkrun_names)):
        if wiki_bool:
            create_parkrun(conn, parkrun_names[i], parkrun_country, parkrun_locations[i], None, None)
        else:
            create_parkrun(conn, parkrun_names[i], parkrun_countries[i], parkrun_locations[i], None, None)

In [11]:
countries = ['USA', 'France']
country_urls = ['https://en.wikipedia.org/wiki/List_of_Parkruns_in_the_United_States_of_America', 'https://en.wikipedia.org/wiki/List_of_Parkruns_in_France']

for i in range(len(countries)):
    add_parkruns_from_country_to_database(countries[i], country_urls[i], True)

In [12]:
add_parkruns_from_country_to_database(None, 'https://samstaggerson.com/2018/01/09/parkrun-alphabet-challenge/', False)

A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in database
A parkrun with that name is already in d

In [14]:
def select_all_parkruns(conn):
    """
    Query all rows in the parkruns table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM parkruns")

    rows = cur.fetchall()

    for row in rows:
        print(row)

In [13]:
select_all_parkruns(conn)

(1, 'Aberbeeg', 'UK', 'Ebbw Valley Cycle Track', -3.144286, 51.711165)
(2, 'Aberdare', 'UK', 'Aberdare Park', -3.460089, 51.718796)
(3, 'Aberdeen', 'UK', 'Aberdeen Beach Esplanade', -2.086256, 57.173898)
(4, 'Aberystwyth juniors', 'UK', 'Plas Crug Park', -4.078402, 52.412885)
(5, 'Aberystwyth', 'UK', 'Plascrug Park', -4.080401, 52.414546)
(6, 'Abingdon', 'UK', 'Rye Meadow, Abingdon', -1.277429, 51.667957)
(7, 'Acton juniors', 'UK', 'Acton Park', -0.258062, 51.507905)
(8, 'Agnew', 'UK', 'Agnew Park', -5.0349, 54.908188)
(9, 'Albert', 'UK', 'Albert Park, Middlesbrough', -1.231788, 54.564972)
(10, 'Aldenham', 'UK', 'Aldenham Country Park', -0.316507, 51.647953)
(11, 'Alderford Lake', 'UK', 'Alderford Lake', -2.675436, 52.953181)
(12, 'Alderman Kneeshaw juniors', 'UK', 'Alderman Kneeshaw Recreation Ground', -0.262291, 53.769454)
(13, 'Alexandra juniors, Moss Side', 'UK', 'Alexandra Park', -2.248631, 53.450725)
(14, 'Alexandra juniors, Oldham', 'UK', 'Alexandra Park', -2.104298, 53.533022)


In [22]:
def select_parkruns_by_country(conn, country):
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM parkruns WHERE parkrun_country='{country}'")

    rows = cur.fetchall()
    
    print(f'The number of parkruns in {country} is {len(rows)}')

    # for row in rows:
    #     print(row)

In [31]:
select_parkruns_by_country(conn, 'UK')
select_parkruns_by_country(conn, 'Australia')
select_parkruns_by_country(conn, 'USA')
select_parkruns_by_country(conn, 'Poland')
select_parkruns_by_country(conn, 'France')
select_parkruns_by_country(conn, 'Germany')

The number of parkruns in UK is 988
The number of parkruns in Australia is 342
The number of parkruns in USA is 48
The number of parkruns in Poland is 59
The number of parkruns in France is 16
The number of parkruns in Germany is 11


In [34]:
def find_parkrun_by_name(conn, parkrun_name):
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM parkruns WHERE parkrun_name='{parkrun_name}'")

    rows = cur.fetchall()

    for row in rows:
        return row

In [37]:
home_parkrun = find_parkrun_by_name(conn, 'Lullingstone')
print(f'My home parkrun is:\n{home_parkrun}')

My home parkrun is:
(538, 'Lullingstone', 'UK', 'Lullingstone Country Park', 0.187147, 51.353118)
